In [118]:
import os
import csv

import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsRegressor
from sklearn.utils import shuffle

In [119]:
data_path = os.path.join(os.path.abspath(os.getcwd()), 'output')

In [120]:
data = pd.read_csv(os.path.join(data_path, 'X[-1.2, 1.2]__Y[1.2, -1.2]__M[1e+28, 1e+29, 1e+30]__N[30000].csv'), names=['x', 'y', 'm', 'score'], delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)

In [121]:
data = shuffle(data).reset_index(drop=True)

In [122]:
training_percent = 70
train = data[:int(len(data)*(training_percent/100))].reset_index(drop=True)
test = data[int(len(data)*(training_percent/100)):].reset_index(drop=True)

In [123]:
X_train = train[['x', 'y', 'm']]
y_train = train['score']

X_test = test[['x', 'y', 'm']]
y_test = test['score']

In [124]:
knn = KNeighborsRegressor(n_neighbors=50, weights='distance', n_jobs=-1)

In [125]:
knn.fit(X_train, y_train)

KNeighborsRegressor(n_jobs=-1, n_neighbors=50, weights='distance')

In [126]:
pred_X = pd.DataFrame([
    [-0.4, 0.3, 1e28], 
    [-0.4, 0.3, 1e29],
    [-0.4, 0.3, 1e30]], columns=['x', 'y', 'm'])

pred_score = pd.DataFrame(knn.predict(pred_X), columns=['pred_score'])
pred_X.join(pred_score)

,x,y,m,pred_score
0,-0.4,0.3,1.000000e+28,0.890660
1,-0.4,0.3,1.000000e+29,0.539209
2,-0.4,0.3,1.000000e+30,0.111595


In [127]:
test_predicted = X_test.join(pd.DataFrame(knn.predict(X_test), columns=['pred_score'])).join(y_test)

In [128]:
test_predicted['error'] = test_predicted.apply(lambda row: abs(row['score'] - row['pred_score']), axis=1)

In [129]:
test_predicted['error'].mean()

0.11459153620084067